In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr

from sklearn.dummy import DummyRegressor

In [3]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 233 entries, Result to InsufficientMaterial
dtypes: float64(111), int64(122)
memory usage: 43.1 MB
None
(24255, 233)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,EndB,EndR,EndK,WhiteСhecks,WhiteQueenCastle,BlackQueenCastle,Promotion,EnPassant,CanClaimDraw,InsufficientMaterial
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.00000,24255.0,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000
mean,0.544877,42.252690,30.881103,11.907442,111.901711,83.967223,-794.591878,988.643950,68.913894,314.678490,...,0.631293,1.03892,1.0,2.015584,1.457308,0.910740,0.064069,0.061637,0.035415,0.000783
std,0.417736,16.235593,233.709202,445.214702,2310.851869,32.489872,2126.099646,2275.840558,64.170646,475.478471,...,0.676858,0.74847,0.0,2.874868,4.179615,3.663225,0.291749,0.245589,0.184831,0.027978
min,0.000000,10.000000,-2162.262500,-11447.000000,-12261.000000,19.000000,-12348.000000,-51.000000,0.000000,7.795716,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,31.000000,-60.758665,0.000000,-362.000000,61.000000,-482.000000,67.000000,30.000000,61.336328,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,40.000000,24.563636,0.000000,0.000000,79.000000,-75.000000,156.000000,49.000000,145.621548,...,1.000000,1.00000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,51.000000,123.987715,26.000000,546.500000,102.000000,-18.000000,700.000000,84.000000,291.317992,...,1.000000,2.00000,1.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,165.000000,2073.289256,11259.000000,12337.000000,329.000000,26.000000,12352.000000,784.000000,3962.575036,...,2.000000,2.00000,1.0,71.000000,35.000000,36.000000,4.000000,2.000000,1.000000,1.000000


In [6]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [7]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [20]:
#mean, median, quantile (0-1)
start = time.time()
dw = DummyRegressor('mean')
db = DummyRegressor('mean')
dw.fit(features, target_white)
db.fit(features, target_black)
white_pred = dw.predict(features)
black_pred =db.predict(features)
full_time=round((time.time() - start)/2,3)
print(full_time, 'c')

0.001 c


In [21]:
print('Для белых:')
maew=mae(target_white,white_pred)
print ('MAE = {:.3f}'.format(maew))
rmsew = (mse(target_white, white_pred))**0.5
print ('RMSE = {:.3f}'.format(rmsew))
corr_coefw = pearsonr(target_white, white_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefw[0]))
print('\nДля черных:')
maeb=mae(target_black,black_pred)
print ('MAE = {:.3f}'.format(maeb))
rmseb = (mse(target_black,black_pred))**0.5
print ('RMSE = {:.3f}'.format(rmseb))
corr_coefb = pearsonr(target_black, black_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefb[0]))

Для белых:
MAE = 215.871
RMSE = 268.406
Correlation coefficient = 0.000

Для черных:
MAE = 218.959
RMSE = 271.197
Correlation coefficient = nan


C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [22]:
pred_elos=list(zip(white_pred,black_pred))
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos),find_root_mean_squared_error(pred_elos,elos)

(10546801.34, 217.415, 269.805)

In [23]:
pred_elos

[(2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189857761286, 2240.0866213151926),
 (2245.7189

In [18]:
#mean, median, quantile (0-1)
start = time.time()
dw = DummyRegressor('median')
db = DummyRegressor('median')
dw.fit(features, target_white)
db.fit(features, target_black)
white_pred = dw.predict(features)
black_pred =db.predict(features)
full_time=round((time.time() - start)/2,3)
print(full_time, 'c')

0.001 c


In [12]:
print('Для белых:')
maew=mae(target_white,white_pred)
print ('MAE = {:.3f}'.format(maew))
rmsew = (mse(target_white, white_pred))**0.5
print ('RMSE = {:.3f}'.format(rmsew))
corr_coefw = pearsonr(target_white, white_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefw[0]))
print('\nДля черных:')
maeb=mae(target_black,black_pred)
print ('MAE = {:.3f}'.format(maeb))
rmseb = (mse(target_black,black_pred))**0.5
print ('RMSE = {:.3f}'.format(rmseb))
corr_coefb = pearsonr(target_black, black_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefb[0]))

Для белых:
MAE = 215.008
RMSE = 269.502
Correlation coefficient = nan

Для черных:
MAE = 218.080
RMSE = 272.249
Correlation coefficient = nan


C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [13]:
#mean, median, quantile (0-1)
start = time.time()
dw = DummyRegressor('quantile',quantile=0.75)
db = DummyRegressor('quantile',quantile=0.75)
dw.fit(features, target_white)
db.fit(features, target_black)
white_pred = dw.predict(features)
black_pred =db.predict(features)
full_time=round((time.time() - start)/2,3)
print(full_time, 'c')

0.001 c


In [14]:
print('Для белых:')
maew=mae(target_white,white_pred)
print ('MAE = {:.3f}'.format(maew))
rmsew = (mse(target_white, white_pred))**0.5
print ('RMSE = {:.3f}'.format(rmsew))
corr_coefw = pearsonr(target_white, white_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefw[0]))
print('\nДля черных:')
maeb=mae(target_black,black_pred)
print ('MAE = {:.3f}'.format(maeb))
rmseb = (mse(target_black,black_pred))**0.5
print ('RMSE = {:.3f}'.format(rmseb))
corr_coefb = pearsonr(target_black, black_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefb[0]))

Для белых:
MAE = 258.944
RMSE = 334.894
Correlation coefficient = nan

Для черных:
MAE = 263.400
RMSE = 340.209
Correlation coefficient = nan


C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [15]:
#mean, median, quantile (0-1)
start = time.time()
dw = DummyRegressor('quantile',quantile=0.25)
db = DummyRegressor('quantile',quantile=0.25)
dw.fit(features, target_white)
db.fit(features, target_black)
white_pred = dw.predict(features)
black_pred =db.predict(features)
full_time=round((time.time() - start)/2,3)
print(full_time, 'c')

0.001 c


In [16]:
print('Для белых:')
maew=mae(target_white,white_pred)
print ('MAE = {:.3f}'.format(maew))
rmsew = (mse(target_white, white_pred))**0.5
print ('RMSE = {:.3f}'.format(rmsew))
corr_coefw = pearsonr(target_white, white_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefw[0]))
print('\nДля черных:')
maeb=mae(target_black,black_pred)
print ('MAE = {:.3f}'.format(maeb))
rmseb = (mse(target_black,black_pred))**0.5
print ('RMSE = {:.3f}'.format(rmseb))
corr_coefb = pearsonr(target_black, black_pred)
print("Correlation coefficient = {:.3f}".format(corr_coefb[0]))

Для белых:
MAE = 267.464
RMSE = 319.176
Correlation coefficient = nan

Для черных:
MAE = 271.065
RMSE = 323.348
Correlation coefficient = nan


C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\Asus\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [26]:
max(elos.WhiteElo)-min(elos.WhiteElo), max(elos.BlackElo)-min(elos.BlackElo)

(1646, 1675)